<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classficiation-CNN/Preprocessing_for_Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [3]:
!pip install opencv-python-headless matplotlib pandas SimpleITK
import os
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom
import matplotlib.pyplot as plt
import SimpleITK as sitk
from google.colab import drive
from sklearn.model_selection import train_test_split
import zipfile
import gc

Mount Google Drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Folder Structure Setup

In [5]:
# Paths
extracted_dataset_path = "/content/drive/MyDrive/DSGP/DSGP_dataset"
base_dir = "/content/drive/MyDrive/DSGP/Preprocessed Dataset"

# Unzip dataset if not already extracted
if not os.path.exists(extracted_dataset_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dataset_path)
else:
    print("Dataset already extracted.")

# Ensure extracted dataset contains files before proceeding
if not any(os.scandir(extracted_dataset_path)):
    raise RuntimeError(f"Error: Extracted dataset is empty at {extracted_dataset_path}")

# Create required folder structure
folders = [
    'Images/Train', 'Images/Val', 'Images/Test',
    'Annotations/Train', 'Annotations/Val', 'Annotations/Test'
]
for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

Dataset already extracted.


Annotation Creation

In [6]:
def create_xml(image_path, class_name, box, size):
    root = ET.Element("annotation")
    ET.SubElement(root, "filename").text = os.path.basename(image_path)
    size_elem = ET.SubElement(root, "size")
    ET.SubElement(size_elem, "width").text = str(size[1])
    ET.SubElement(size_elem, "height").text = str(size[0])
    ET.SubElement(size_elem, "depth").text = "3"
    obj = ET.SubElement(root, "object")
    ET.SubElement(obj, "name").text = class_name
    ET.SubElement(obj, "pose").text = "Unspecified"
    bndbox = ET.SubElement(obj, "bndbox")
    ET.SubElement(bndbox, "xmin").text = str(box[0])
    ET.SubElement(bndbox, "ymin").text = str(box[1])
    ET.SubElement(bndbox, "xmax").text = str(box[2])
    ET.SubElement(bndbox, "ymax").text = str(box[3])
    return minidom.parseString(ET.tostring(root)).toprettyxml()

def find_tumor_bbox(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        return [x, y, x + w, y + h]
    return [0, 0, 0, 0]


Preprocessing Functions

In [7]:
# Skull Stripping
def skull_stripping(img):
    sitk_img = sitk.GetImageFromArray(img)
    sitk_img = sitk.Cast(sitk_img, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(sitk_img)
    stripped_img = sitk.GetArrayFromImage(sitk.Mask(sitk_img, mask))
    return cv2.normalize(stripped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

In [8]:
def normalize_image(img):
    p2, p98 = np.percentile(img, (2, 98))
    img = np.clip(img, p2, p98)
    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

In [9]:
def preprocess_pipeline(img_path, target_size=(256, 256)):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

    if img is None:
        print(f"Warning: Could not read image {img_path}. Skipping...")
        return None

    if img.dtype == np.float64:
        img = (img * 255).astype(np.uint8)

    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    skull_free = skull_stripping(img)
    normalized = normalize_image(skull_free)

    final_img = (normalized * 255).astype(np.uint8)
    return cv2.resize(final_img, target_size)

Data Augmentation

In [10]:
def augment_image(img):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), np.random.randint(-15, 15), np.random.uniform(0.9, 1.1))
    return cv2.warpAffine(img, M, (w, h))

Preprocessing Function

In [11]:
def process_dataset():
    data = []
    for root, dirs, files in os.walk(extracted_dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                class_label = os.path.basename(root)
                if "no tumor" in class_label.lower():
                    class_label = "No_Tumor"
                data.append({"path": os.path.join(root, file), "class": class_label})

    df = pd.DataFrame(data)

    # Class Balancing (Upsampling)
    min_samples = max(df['class'].value_counts())  # Balance by increasing small classes
    balanced_df = df.groupby('class', group_keys=False, observed=True).apply(lambda x: x.sample(min_samples, replace=True)).reset_index(drop=True)

    # Splitting
    train_df, temp_df = train_test_split(balanced_df, test_size=0.3, stratify=balanced_df['class'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['class'])

    # Processing
    for split_df, split_name in zip([train_df, val_df, test_df], ['Train', 'Val', 'Test']):
        for idx, row in split_df.iterrows():
            processed_img = preprocess_pipeline(row['path'])
            if processed_img is None:
                continue

            if row['class'] != "No_Tumor" and split_name == "Train":
                processed_img = augment_image(processed_img)

            img_filename = f"{split_name.lower()}_{idx}.png"
            class_folder = f"{base_dir}/Images/{split_name}/{row['class']}"
            os.makedirs(class_folder, exist_ok=True)
            cv2.imwrite(os.path.join(class_folder, img_filename), cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))

Run Preprocessing

In [12]:
process_dataset()
print("Preprocessing complete! All files saved in:", base_dir)

<ipython-input-11-77a6861c71b8>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby('class', group_keys=False, observed=True).apply(lambda x: x.sample(min_samples, replace=True)).reset_index(drop=True)


Preprocessing complete! All files saved in: /content/drive/MyDrive/DSGP/Preprocessed Dataset
